In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import sklearn

%matplotlib inline

In [ ]:
data = pd.read_csv("../input/loan.csv")
data.head(10)

In [ ]:
data.shape

In [ ]:
data.columns.sort_values()

# Analysis of loan status

In [ ]:
data.loan_status.unique()

In [ ]:
data.loan_status.value_counts()

*Issued* is not related to our predictions since it does not talk about fradulent behaviour. Similarly *In Grace Period* does not give information about fradulent behaviour since we don't know the future outcome.

In [ ]:
# DF Transformed
data = data[~data.loan_status.isin(["Issued", "In Grace Period"])]

In [ ]:
data.loan_status.value_counts()

In [ ]:
data.shape

In [ ]:
bad_status = ["Charged Off", "Default", "Late (31-120 days)", "Late (16-30 days)", "Does not meet the credit policy. Status:Charged Off"]
good_status = ["Fully Paid", "Current", "Does not meet the credit policy. Status:Fully Paid"]

In [ ]:
data["defaulter"] = data.loan_status.apply(lambda x: 1 if x in good_status else -1)
data.head()

In [ ]:
data.defaulter.value_counts()

We have thus converted loan_status into a label called *defaulter* which we wish to predict. We have dropped *Issued* and *In Grace Period* for the reasons mentioned above.

# Filtering columns

The dataset contains a large number of columns, not all of which might be relevant. We try and remove certain columns that we feel does not contribute in any way to the predictions. The column descriptions are given in the data dictionary.

In [ ]:
data.annual_inc.isna().sum()

In [ ]:
data[data.annual_inc.isna()][["annual_inc", "annual_inc_joint", "defaulter"]]

In [ ]:
# DF Transformed
temp = data.shape[0]
data = data[data.annual_inc.notnull()]
print("Dropped:", temp - data.shape[0])

In [ ]:
num_inc_joint_na = data.annual_inc_joint.isna().sum()
num_inc_joint_na

In [ ]:
inc_df = data[data.annual_inc_joint.notnull()][["annual_inc", "annual_inc_joint", "defaulter"]]
inc_df["joint_diff"] =  inc_df.annual_inc_joint - inc_df.annual_inc
inc_df.head(10)

In [ ]:
inc_df.joint_diff.describe()

In [ ]:
data.shape[0] - num_inc_joint_na

In [ ]:
inc_df.groupby("defaulter").joint_diff.count()

Therefore *annual_inc_joint* is available only for 511 rows. Every row that contains *annual_inc_joint*  also contains *annual_inc* and the values differ significantly. Moreover only 2 cases are defaulters. It makes sense to drop the column due to its occurence and contribution to our predictions.

In [ ]:
data.count().sort_values()

In [ ]:
# DF Transformed
min_size = data.shape[0]*0.9
print(data.shape)
data = data.dropna(thresh=min_size, axis=1)
print(data.shape)

Removed columns with more than 10% null values.

In [ ]:
data.columns.sort_values()

In [ ]:
lst_to_be_removed = ['collection_recovery_fee', 'earliest_cr_line', 'funded_amnt_inv', 'inq_last_6mths', 'issue_d', 'last_credit_pull_d', 'last_pymnt_amnt', 'last_pymnt_d', 'member_id', 'out_prncp_inv', 'recoveries', 'title', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'url']

In [ ]:
# DF Transformed
data.drop(lst_to_be_removed, axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data.shape

After analysing from the data-dictionary we removed columns that we felt weren't related to the output. We also removed columns that won't be available at the time of loan application.

# Analysis of attributes

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(x='defaulter', data=data)
plt.show()

In [ ]:
data.defaulter.value_counts()

Extremely skewed dataset. Might create problems when building models.

In [ ]:
data.columns.sort_values()

In [ ]:
to_remove = []

## collection_12_ex_med

In [ ]:
data.collections_12_mths_ex_med.value_counts()

In [ ]:
temp = data[data['collections_12_mths_ex_med'] > 0]
temp.shape[0]

In [ ]:
len(temp[temp.defaulter == -1])

In [ ]:
to_remove.append("collections_12_mths_ex_med")

Completely skewed attribute. Doesn't seem to contribute to output.

## dti (Debt-to-income ratio)

In [ ]:
data.groupby('defaulter').dti.mean()

In [ ]:
data.groupby('defaulter').dti.std()

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot("dti", data=data)
plt.show()

In [ ]:
data.dti.describe()

In [ ]:
temp = data[data.dti < 100] # just greater than 75th percentile
plt.figure(figsize=(8,4))
sns.boxplot("dti", data=temp)
plt.show()

In [ ]:
data.shape[0] - temp.shape[0]

In [ ]:
temp.defaulter.value_counts()

In [ ]:
temp.groupby("defaulter").dti.mean()

In [ ]:
temp.groupby("defaulter").dti.std()

The *dti* columns doesn't seem to contribute although logically it represents a good measure of loan repayment. The mean values are almost identical for both cases of defaulter. We can maybe analyze it by using it for building models.

# funded_amnt

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot("funded_amnt", data=data)
plt.show()

In [ ]:
data.groupby("defaulter").funded_amnt.mean()

In [ ]:
data.groupby("defaulter").funded_amnt.std()

In [ ]:
plt.figure(figsize=(8,8))
sns.violinplot(x='defaulter', y='funded_amnt', data=data)
plt.show()

In [ ]:
to_remove.append('funded_amnt')

Values seem to be distributed similarly across both defaulter values. Thus it does not seem to be contributing to the result.

## initial_list_status

In [ ]:
data.groupby('defaulter').initial_list_status.value_counts()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='initial_list_status', hue='defaulter', data=data)
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('initial_list_status', fontsize=12)
plt.show()

Probability of being a defaulter is slightly more incase of fractional loans ('f') than whole loans ('w'). It seems to be contributing slightly. This feature seems to be useful considering our skewed dataset.

## int_rate (Interest rate)

In [ ]:
plt.figure(figsize=(8,6))
sns.violinplot(x='defaulter', y='int_rate', data=data)
plt.show()

In [ ]:
data.groupby("defaulter").int_rate.mean()

In [ ]:
data.groupby("defaulter").int_rate.std()

This feature seems important since a higher interest rate seems to lead to more defaulters.

## policy_code

In [ ]:
data.policy_code.value_counts()

In [ ]:
to_remove.append("policy_code")

Clearly not useful since it has only one value for all columns.

# pub_rec

In [ ]:
data.pub_rec.value_counts()

In [ ]:
data.groupby("pub_rec").defaulter.value_counts()

In [ ]:
to_remove.append("pub_rec")

The probability of a defaulter given the pub_rec value doesn't seem to be much different than just the probability of the class. The higher values having defaulter as 1 might be because of the skewed data set.

## pymnt_plan

In [ ]:
data.pymnt_plan.value_counts()

In [ ]:
data.groupby("pymnt_plan").defaulter.value_counts()

In [ ]:
to_remove.append("pymnt_plan")

The attribute values are skewed and the distribution of defaulters is also uniform. This attribute doesn't seem to be useful.

## revol_util

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot("revol_util", data=data)
plt.show()

In [ ]:
temp = data[data.revol_util < 120] # Just near whisker end

plt.figure(figsize=(8,4))
sns.violinplot(x='defaulter', y='revol_util', data=temp) # We use 
plt.show()

In [ ]:
data.groupby("defaulter").revol_util.mean()

In [ ]:
data.groupby("defaulter").revol_util.std()

In [ ]:
to_remove.append("revol_util")

Even this attribute doesn't seem to affect the condtional distribution of defaulter.

## sub_grade

In [ ]:
len(data.sub_grade.unique())

In [ ]:
data.sub_grade.value_counts()

In [ ]:
data.groupby("sub_grade").defaulter.value_counts()

In [ ]:
data.groupby("grade").sub_grade.value_counts()

In [ ]:
temp = pd.crosstab(data.sub_grade, data.defaulter)
temp.head()

In [ ]:
temp["ratio"] = temp[-1] / temp[1]
temp.head()

In [ ]:
temp["sub_grade"] = temp.index
temp.head()

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(x='sub_grade', y="ratio", data=temp)
plt.ylabel('Ratio of defaulters to non-defaulters', fontsize=12)
plt.xlabel('sub_grade', fontsize=12)
plt.show()

sub_grade seems to be an important column since the ratio of defaulters to non-defaulters increases with decrease in subgrade. We can compare it with grade when we build models, maybe with respect to measures like info gain.

## application_type

In [ ]:
data.groupby("application_type").defaulter.value_counts()

In [ ]:
to_remove.append("application_type")

There doesn't seem to be any relation between defaulter and application_type.

## delinq_2yrs

In [ ]:
len(data.delinq_2yrs.unique())

In [ ]:
data.groupby("delinq_2yrs").defaulter.value_counts()

In [ ]:
temp = pd.crosstab(data.delinq_2yrs, data.defaulter)
temp.head()

In [ ]:
temp["ratio"] = temp.apply(lambda x: (x[-1]/x[1]), axis=1)
temp.head()

In [ ]:
temp["delinq"] = temp.index
plt.figure(figsize=(14,6))
sns.barplot(x='delinq', y="ratio", data=temp)
plt.ylabel('Ratio of defaulters to non-defaulters', fontsize=12)
plt.xlabel('delinq_2yrs', fontsize=12)
plt.show()

In [ ]:
to_remove.append("delinq_2yrs")

The probability of a defaulter conditioned on the delinq_2yrs value is almost same as just the probability of a defaulter suggesting that the gain is less. Wherever the probability is high the number of instances are extremely low i.e. around 1-10

## emp_title

In [ ]:
len(data.emp_title.unique())

In [ ]:
data.emp_title.value_counts()

In [ ]:
data[data.emp_title.isna()].defaulter.value_counts()

The null rows can't be dropped due to high number of defaulters. But the column logically contributes to the final output.

## purpose

In [ ]:
data.purpose.value_counts()

In [ ]:
data.groupby('purpose').defaulter.value_counts()

In [ ]:
temp = pd.crosstab(data.purpose, data.defaulter)
temp["ratio"] = temp.apply(lambda x: (x[-1]/x[1]), axis=1)
temp["purpose"] = temp.index
temp.head()

In [ ]:
plt.figure(figsize=(16,6))
sns.barplot(x='purpose', y="ratio", data=temp, order=temp.sort_values("ratio").purpose.values)
plt.ylabel('Ratio of defaulters to non-defaulters', fontsize=12)
plt.xlabel('purpose', fontsize=12)
plt.show()

The ratio of defaulters conditioned on the purpose value is quite high which indicates that this is a good column.

## revol_bal

In [ ]:
plt.figure(figsize=(8,4))
sns.violinplot("revol_bal", data=data) # We use 
plt.show()

In [ ]:
data.revol_bal.describe()

In [ ]:
data.groupby("defaulter").revol_bal.mean()

In [ ]:
data.groupby("defaulter").revol_bal.std()

In [ ]:
plt.figure(figsize=(8,4))
sns.violinplot(y="revol_bal", x="defaulter", data=data[data.revol_bal < 50000]) # We remove outliers in the plot
plt.show()

In [ ]:
to_remove.append("revol_bal")

There doesn't seem to be any difference in the distribution. This feature doesn't seem too useful.

# Short Summary

In [ ]:
to_remove.append("loan_status")
print(to_remove)

In [ ]:
data.isna().sum()

In [ ]:
y = len(data[data.defaulter == -1])

def defaulter_in_null(df, attr):
    temp = df[df[attr].isna()]
    x = len(temp[df.defaulter == -1])
    return (x, x/len(temp), x/y)
    
attrs = [
    "total_rev_hi_lim",
    "tot_cur_bal",
    "tot_coll_amt",
    "acc_now_delinq",
    "collections_12_mths_ex_med",
    "total_acc",
    "revol_util",
    "pub_rec",
    "open_acc",
    "delinq_2yrs",
    "emp_length",
    "emp_title"
]

for i in attrs:
    print(defaulter_in_null(data, i))

We'll need to handle these null values in a proper way.

## term

In [ ]:
data.term = data.term.apply(lambda x: int(re.search(r'\d+', x).group()))

In [ ]:
data.groupby("term").defaulter.value_counts()

Once again there doesn't seem to be an increase in the probability of a defaulter conditioned on the value of *term*.

## verification_status

In [ ]:
data.groupby("verification_status").defaulter.value_counts()

In [ ]:
data.emp_length.unique()

In [ ]:
temp = pd.crosstab(data.verification_status, data.defaulter)
temp["ratio"] = temp.apply(lambda x: (x[-1]/x[1]), axis=1)
temp["vf"] = temp.index
temp.head()

In [ ]:
plt.figure(figsize=(16,6))
sns.barplot(x='vf', y="ratio", data=temp, order=temp.sort_values("ratio").vf.values)
plt.ylabel('Ratio of defaulters to non-defaulters', fontsize=12)
plt.xlabel('Verification Status', fontsize=12)
plt.show()

Considering that the ratio of defaulters to non-defaulters is 0.7 we don't gain much information by conditioning on verification_status.

# Train Test Split

In [ ]:
train, test = sklearn.model_selection.train_test_split(data, random_state=100, shuffle=True, test_size=0.3)

In [ ]:
print("Train:", train.shape)
print("Test:", test.shape)

In [ ]:
print("Train Defaulters:", len(train[train.defaulter == -1]))
print("Test Defaulters:", len(test[test.defaulter == -1]))

In [ ]:
train.to_csv("train.csv")
test.to_csv("test.csv")